# Instalando bibliotecas

In [ ]:
%pip install numpy pandas tensorflow scikit-learn matplotlib

# Importando bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
import matplotlib.pyplot as plt

# --- 1. Pré-processamento dos Dados ---

In [ ]:
# Carregar o dataset. O arquivo original não tem cabeçalho, então precisamos defini-lo.
# Os dados devem ser lidos com a codificação 'ISO-8859-1'.
df = pd.read_csv('sentiment140.csv', encoding='ISO-8859-1', header=None)

# O dataset tem 6 colunas. A coluna 0 é o rótulo de sentimento e a 5 é o texto.
# Rótulo 0: Negativo, Rótulo 4: Positivo.
# Vamos mapear os rótulos para 0 (negativo) e 1 (positivo) para a classificação binária.
df = df.iloc[:, [0, 5]]
df.columns = ['target', 'text']
df['target'] = df['target'].replace({0: 0, 4: 1})

# Amostragem para reduzir o tempo de processamento.
# Pegamos uma amostra aleatória de 50.000 tweets.
df = df.sample(n=50000, random_state=24).reset_index(drop=True)

# Função de limpeza do texto
def clean_text(text):
    text = text.lower()  # Converter para minúsculas
    text = re.sub("https?://\\S+|www\\.\\S+", '', text)  # Remover URLs
    text = re.sub('@\\S+', '', text)  # Remover menções (@usuario)
    text = re.sub('#', '', text)  # Remover o símbolo da hashtag
    text = re.sub('[\\W\\d_]+', ' ', text)  # Remover caracteres especiais, números e underscores
    return text

df['text'] = df['text'].apply(clean_text)

# Tokenização e Sequenciamento
# Criar um vocabulário com as palavras
max_words = 10000
tokenizer = Tokenizer(num_words=max_words, oov_token="<unk>")
tokenizer.fit_on_texts(df['text'])
word_index = tokenizer.word_index

# Converter o texto em sequências de números
X = tokenizer.texts_to_sequences(df['text'])

# Padding para garantir que todas as sequências tenham o mesmo tamanho
max_len = 50
X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post')

# Preparar os rótulos (Y)
y = df['target'].values

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24, stratify=y)

# --- 2. Implementação da RNN (LSTM) ---

In [ ]:
#Configurações
vocab_size = len(word_index) + 1
embedding_dim = 128
lstm_units = 64

# Construção do modelo sequencial
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Dropout(0.2))  # Dropout para regularização
model.add(LSTM(lstm_units))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#model.summary()

# --- 3. Treinamento do Modelo ---

In [ ]:
# Treinar o modelo e salvar o histórico para plotar as curvas de erro
epochs = 15
batch_size = 64

history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test))

# --- 4. Avaliação e Resultados ---

# Avaliar o desempenho no conjunto de teste
loss, accuracy = model.evaluate(X_test, y_test)
print(f'\nPerda (Loss) no conjunto de teste: {loss:.4f}')
print(f'Acurácia (Accuracy) no conjunto de teste: {accuracy:.4f}')

# Plotar as curvas de acurácia e perda
plt.figure(figsize=(12, 5))

# Plot da Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Acurácia de Treino')
plt.plot(history.history['val_accuracy'], label='Acurácia de Validação')
plt.title('Curva de Acurácia')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()
plt.grid(True)

# Plot da Perda (Loss)
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Perda de Treino')
plt.plot(history.history['val_loss'], label='Perda de Validação')
plt.title('Curva de Perda')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
plt.savefig("plot.png")
plt.close()

# --- 4. Exemplos de Predições ---

In [ ]:
# Selecionar 5 exemplos do conjunto de teste
# Dividir o DataFrame de textos também
text_train, text_test, _, _ = train_test_split(df['text'], y, test_size=0.2, random_state=24, stratify=y)

sample_tweets = X_test[:5]
true_labels = y_test[:5]
sample_predictions = (model.predict(sample_tweets) > 0.5).astype("int32").flatten()

# Acessar os textos originais do novo DataFrame de teste
text_test_samples = text_test.iloc[:5].values

print("\n--- Exemplos de Predições ---")
for i in range(5):
    prediction_label = "Positivo" if sample_predictions[i] == 1 else "Negativo"
    true_label = "Positivo" if true_labels[i] == 1 else "Negativo"
    
    result = "✅ Correto" if sample_predictions[i] == true_labels[i] else "❌ Incorreto"
    
    print(f"\nTweet: {text_test_samples[i]}")
    print(f"Predição: {prediction_label}")
    print(f"Rótulo Verdadeiro: {true_label}")
    print(f"Resultado: {result}")